In [2]:

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy import stats 
!pip install statsmodels
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')



In [3]:
#setting the layout 

sns.set_style("whitegrid") 
plt.rcParams["figure.figsize"] = (10, 6) 
plt.rcParams["font.size"] = 11

print("="*70)
print("PRICE ELASTICITY ANALYSIS - BIG MART RETAIL DATA")
print("="*70)

PRICE ELASTICITY ANALYSIS - BIG MART RETAIL DATA


In [4]:
df = pd.read_csv("Train (2).csv")

df.shape


(8523, 12)

In [5]:
#Renaming columns for clearer analysis 

df = df.rename(columns = { 
    'Item_MRP' : 'price', 
    'Item_Outlet_Sales' : 'sales', 
    'Item_Type' : 'category' }) 

#storing the original size for cleaning impact 
original_size = len(df) 

In [6]:
#dropping null or missing values 
df1 = df.dropna(subset = ['price', 'sales']) 
# droppinp zero or negative prices are data quality issues in retail context 

df = df[(df['price'] > 0) & (df['sales'] > 0)] 
cleaned_size = len(df1) 
removed_records = original_size - cleaned_size 



In [42]:
print(f"[INFO] Records removed: {removed_records} ({removed_records/original_size*100:.1f}%)")
print(f"[INFO] Clean dataset size: {cleaned_size:,} records")
print(f"[INFO] Number of product categories: {df['category'].nunique()}")
print(f"[INFO] Price range: Rs {df['price'].min():.2f} - ₹{df['price'].max():.2f}")
print(f"[INFO] Sales range: Rs {df['sales'].min():.2f} - {df['sales'].max():.2f}")

[INFO] Records removed: 0 (0.0%)
[INFO] Clean dataset size: 8,523 records
[INFO] Number of product categories: 16
[INFO] Price range: Rs 31.29 - ₹266.89
[INFO] Sales range: Rs 33.29 - 13086.96


In [13]:
#finding the linear relationship strength between price and sales 

correlation = df['price'].corr(df['sales']) 
print(f"Overall Price-Sales Correlation: {correlation:.3f}")

#intepretting the correlation 

if abs(correlation) < 0.3: 
    print("Weak correlation - relationship might vary by category") 
elif abs(correlation) < 0.7: 
    print("moderate correlation")
else: 
    print("strong correlation detected")



Overall Price-Sales Correlation: 0.568
moderate correlation


In [14]:
#Calculating price elasticity by product category 

#using log-log ols regression log(sales) = BO + B1*Log(price) + e
#b1 represents price elasticity 

elasticity_results = []

#filtering data for categories to calculate category specific elasticity 
for category in df['category'].unique(): 
    cat_data = df[df['category'] == category].copy()

    #at least 10 observations per variable
    if len(cat_data) < 10:
        print(f"[SKIP] {category}: Insufficient data (n={len(cat_data)})")
        continue
    #log log transformation for elasticity estimation
    #the coefficient directly gives us elasticity
    cat_data['log_price'] = np.log(cat_data['price'])
    cat_data['log_sales'] = np.log(cat_data['sales']) 

    #prepare independent variable(price) with intercept 
    X = sm.add_constant(cat_data['log_price']) #adds intercept with price 
    y = cat_data['log_sales'] 

    try: 
        model = sm.OLS(y, X).fit()
        elasticity = model.params['log_price']#It gives the b1's /slopes of line or elastocity
        #elasticity = dx/dp * p/x or logx/logp 
        p_value = model.pvalues['log_price']
        r_squared = model.rsquared 

        #storing results for this category 
        elasticity_results.append({
            'category' : category, 
            'elasticity': elasticity,
            'p_value': p_value,
            'r_squared': r_squared,
            'avg_price': cat_data['price'].mean(),
            'median_price': cat_data['price'].median(),
            'total_sales': cat_data['sales'].sum(),
            'num_products': len(cat_data)
        })
         # Print progress
        sig_marker = "***" if p_value < 0.01 else "**" if p_value < 0.05 else "*" if p_value < 0.1 else ""
        print(f"[CALC] {category:25s} | Elasticity: {elasticity:6.3f} {sig_marker} | n={len(cat_data):4d} | R²={r_squared:.3f}")
        
    except Exception as e:
        print(f"[ERROR] {category}: Regression failed - {str(e)}")
        continue


[CALC] Dairy                     | Elasticity:  1.064 *** | n= 682 | R²=0.323
[CALC] Soft Drinks               | Elasticity:  0.898 *** | n= 445 | R²=0.270
[CALC] Meat                      | Elasticity:  1.017 *** | n= 425 | R²=0.246
[CALC] Fruits and Vegetables     | Elasticity:  1.026 *** | n=1232 | R²=0.258
[CALC] Household                 | Elasticity:  1.007 *** | n= 910 | R²=0.268
[CALC] Baking Goods              | Elasticity:  1.010 *** | n= 648 | R²=0.257
[CALC] Snack Foods               | Elasticity:  1.007 *** | n=1200 | R²=0.236
[CALC] Frozen Foods              | Elasticity:  1.114 *** | n= 856 | R²=0.362
[CALC] Breakfast                 | Elasticity:  1.223 *** | n= 110 | R²=0.300
[CALC] Health and Hygiene        | Elasticity:  0.982 *** | n= 520 | R²=0.287
[CALC] Hard Drinks               | Elasticity:  1.099 *** | n= 214 | R²=0.366
[CALC] Canned                    | Elasticity:  1.032 *** | n= 649 | R²=0.269
[CALC] Breads                    | Elasticity:  0.968 *** | n= 2

In [34]:
#converting results to dataframe 
elasticity_df = pd.DataFrame(elasticity_results) 

#sorting elasticity from highest to lowest(most inelastic to most elastic)
elasticity_df = elasticity_df.sort_values('elasticity', ascending = False) 

#economic classification based on elasticity magnitude 
#if e < 1 :  inelastic 
# e > 1 : elastic
# e =1 : elastic 

def classify_demand(elasticity): 
    abs_elasticity = abs(elasticity) 
    if abs_elasticity < 0.5: 
        return "Highly Inelastic" 
    elif abs_elasticity < 1: 
        return "Inelastic" 
    elif abs_elasticity == 1: 
        return "Unit Elastic" 
    elif (abs_elasticity > 1) and (abs_elasticity < 1.5):
        return "Elastic"
    else: 
        return "Highly Elastic"

#apply classification 

elasticity_df['demand_type'] = elasticity_df['elasticity'].apply(classify_demand) 

#simplify to binary classification 
elasticity_df['can_increase_price'] = elasticity_df['elasticity'].apply(
    lambda x: 'No' if x > 1 else 'Yes'
)

#summary statistics 
print(elasticity_df['demand_type'].value_counts().to_string())

inelastic_count = (elasticity_df['elasticity'] < 1).sum()
inelastic_pct = (inelastic_count / len(elasticity_df)) * 100 

print(f"\n[KEY FINDING]  {inelastic_pct:.1f}% of categories have inelastic demand") 

print(f"[IMPLICATION] These {inelastic_count} categories are candidates for price optimization")

print(f"Avg Elasticity : {np.mean(elasticity_df['elasticity']):.2f}")


demand_type
Elastic      12
Inelastic     4

[KEY FINDING]  25.0% of categories have inelastic demand
[IMPLICATION] These 4 categories are candidates for price optimization
Avg Elasticity : 1.02


In [16]:
#Visualization 

#price vs sales scatter plot 

plt.figure(figsize=(10, 6)) 
plt.scatter(df['price'], df['sales'], alpha = 0.3, s=20, color='steelblue', edgecolors='none')
plt.xlabel('Price(Rs)', fontsize = 12, fontweight = 'bold') 
plt.ylabel('Sales Volume', fontsize = 12, fontweight = 'bold') 
plt.title('Price vs Sales Relationship - Big mart Retail Data', fontsize = 14, fontweight = 'bold', pad = 20) 

#adding trendline
z = np.polyfit(df['price'], df['sales'], 1) 
p = np.poly1d(z) 
plt.plot(df['price'].sort_values(), p(df['price'].sort_values()), 
         "r--", alpha=0.8, linewidth=2, label=f'Trend line')
plt.legend()
plt.grid("TRUE", alpha = 0.3)
plt.tight_layout()
plt.savefig('price_vs_sales.png', dpi=300, bbox_inches='tight')
plt.close()
plt.show()




In [20]:
#elasticity by category bar chart 

plt.figure(figsize = (12, 8)) 
# Color code by demand type (green = can increase price, red = price sensitive)
colors = ['#27ae60' if x < 1 else '#e74c3c' for x in elasticity_df['elasticity']]

bars = plt.barh(elasticity_df['category'], elasticity_df['elasticity'], color = colors, alpha = 0.8) 

plt.xlabel('Price Elasticity Coefficient', fontsize = 12, fontweight = 'bold') 
plt.ylabel('Product Category', fontsize = 12, fontweight = 'bold') 
plt.title('Price Elasticity by Product Category\nGreen = Price Increase Opportunity | Red = Price Sensitive', 
          fontsize=14, fontweight='bold', pad=20)

# Add reference line at 1 (unit elasticity threshold)
plt.axvline(x=-1, color='black', linestyle='--', linewidth=2, 
            label='Unit Elastic (E = -1)', alpha=0.7)

# Add reference line at 0 (perfectly inelastic - theoretical)
plt.axvline(x=0, color='gray', linestyle=':', linewidth=1, 
            label='Perfectly Inelastic (E = 0)', alpha=0.5)
plt.legend(loc='lower right', fontsize=10)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('elasticity_by_category.png', dpi=300, bbox_inches='tight')
plt.close()
plt.show()



In [21]:
#demand type distribution pie chart 
plt.figure(figsize = (10, 8)) 

simple_classification = elasticity_df['elasticity'].apply(
    lambda x : 'Inelastic\n (can increase price)' if x < 1 else 'Elastic\n (price sensitive)')


<Figure size 1000x800 with 0 Axes>

In [22]:

#creating pie charts 
demand_counts = simple_classification.value_counts()

colors_pie = ['#e74c3c','#27ae60']

explode = (0.05, 0)  # Slightly separate the inelastic slice for emphasis

wedges, texts, autotexts = plt.pie(
    demand_counts, 
    labels=demand_counts.index, 
    autopct='%1.1f%%',
    colors=colors_pie, 
    startangle=90, 
    explode=explode,
    textprops={'fontsize': 12, 'fontweight': 'bold'},
    shadow=True
)

# Make percentage text more readable
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontsize(14)

plt.title('Product Category Distribution by Price Sensitivity\n(Total Categories: {})'.format(len(elasticity_df)), 
          fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig('demand_type_distribution.png', dpi=300, bbox_inches='tight')
plt.close()
plt.show()


In [23]:
#Buisness insights and recommendations 
inelastic = elasticity_df[elasticity_df['elasticity'] < 1].sort_values('avg_price', ascending = False) 

elastic = elasticity_df[elasticity_df['elasticity'] > 1].sort_values('elasticity') 

#Insight 1 : Price Increase oppurtunities (Inelastic Products) 
print(f" PRICE INCREASE OPPORTUNITIES ({len(inelastic)} categories)")
print("\n[STRATEGY] These categories show inelastic demand - customers will continue")
print("           buying even with moderate price increases (5-10%)")
print("\n[RECOMMENDATION] Implement strategic price increases to improve margins")

 PRICE INCREASE OPPORTUNITIES (4 categories)

[STRATEGY] These categories show inelastic demand - customers will continue
           buying even with moderate price increases (5-10%)

[RECOMMENDATION] Implement strategic price increases to improve margins


In [41]:
if len(inelastic) > 0: 
    print("\n" + inelastic[['category', 'elasticity', 'avg_price', 'total_sales']].to_string(index = False))

    print(f"\n[TOP OPPORTUNITY] {inelastic.iloc[0]['category']}") 
    print(f" - Elasticity : {inelastic.iloc[0]['elasticity']:.3f} (highly inelastic)") 
    print(f" - Average Price : Rs{inelastic.iloc[0]['avg_price']:.2f}") 
    print(f" - Total Sales : Rs {inelastic.iloc[0]['total_sales']:.2f}")
else: 
     print("\n[NOTE] No strongly inelastic categories found in current dataset")





          category  elasticity  avg_price  total_sales
           Seafood    0.864431 141.841719  148868.2194
            Breads    0.968193 140.952669  553237.1888
       Soft Drinks    0.898465 131.492506  892897.7220
Health and Hygiene    0.981817 130.818921 1045200.1378

[TOP OPPORTUNITY] Seafood
 - Elasticity : 0.864 (highly inelastic)
 - Average Price : Rs141.84
 - Total Sales : Rs 148868.22


In [25]:
# Price Sensitivity Products (Elastic Products) 
print(f"\n{'='*70}")
print(f"  PRICE SENSITIVE CATEGORIES ({len(elastic)} categories)")
print(f"{'='*70}")
print("\n[STRATEGY] These categories show elastic demand - price increases will")
print("           significantly reduce sales volume")
print("\n[RECOMMENDATION] Maintain competitive pricing, focus on volume and promotions")




  PRICE SENSITIVE CATEGORIES (12 categories)

[STRATEGY] These categories show elastic demand - price increases will
           significantly reduce sales volume

[RECOMMENDATION] Maintain competitive pricing, focus on volume and promotions


In [37]:
if len(elastic) > 0:
    # Show top 5 most elastic for brevity
    display_elastic = elastic.head(5)
    print("\n" + display_elastic[['category', 'elasticity', 'avg_price', 'total_sales']].to_string(index=False))
    
    if len(elastic) > 5:
        print(f"\n[NOTE] Showing top 5 of {len(elastic)} elastic categories")


    category  elasticity  avg_price  total_sales
   Household    1.007212 149.424753 2055493.7132
 Snack Foods    1.007431 146.194934 2732786.0870
Baking Goods    1.010481 126.380766 1265525.3422
        Meat    1.017023 139.882032  917565.6120
      Others    1.025264 132.851430  325517.6096

[NOTE] Showing top 5 of 12 elastic categories


In [1]:
print(f"\n{'='*70}")
print("REVENUE IMPACT SIMULATION")
print(f"{'='*70}")


REVENUE IMPACT SIMULATION


In [27]:
print("\n[SCENARIO] Simulating 5% price increase on inelastic product categories")
print("[FORMULA] Revenue Change = (1 + price_change) × (1 + quantity_change) - 1")
print("          where quantity_change = elasticity × price_change")


[SCENARIO] Simulating 5% price increase on inelastic product categories
[FORMULA] Revenue Change = (1 + price_change) × (1 + quantity_change) - 1
          where quantity_change = elasticity × price_change


In [38]:
if len(inelastic) > 0 : 
    #simulate for top 3 inelastic categories 
    simulation_count = min(3, len(inelastic)) 
for i, (_,row) in enumerate(inelastic.head(simulation_count).iterrows(), 1) : 
    price_increase = 0.05 #5% increase 

    #Calculate expected quantity change using elasticity formula
    #Elasticity = %change in qty / % change in price
    #Therefore % change in quantity = elasticity * %change in price 
    quantity_change = row['elasticity'] * price_increase 

    #calculate total revenue impact
    #new revenue = (new price) * (new qty) 
    #              (1.05 * old pricee) * (1 + quantity_change) * old_quantity
    #              (1.05) * ( 1 + qty_change_ * old revenue 
    revenue_change = (1 + price_increase) * (1 + quantity_change) - 1

    #estimated dollar impact 
    estimated_revenue_impact = row['total_sales'] * revenue_change
    print(f"\n[SIMULATION {i}] {row['category']}")
    print(f"  Current Average Price: Rs{row['avg_price']:.2f}")
    print(f"  New Price (+ 5%):      Rs{row['avg_price'] * 1.05:.2f}")
    print(f"  Elasticity:            {row['elasticity']:.3f}")
    print(f"  Expected Qty Change:   {quantity_change*100:+.2f}%")
    print(f"  Expected Revenue Change: {revenue_change*100:+.2f}%")
    print(f"  Estimated ₹ Impact:    Rs{estimated_revenue_impact:+,.2f}")

    # Business interpretation
    if revenue_change > 0:
        print(f"  [INSIGHT] Price increase likely to improve revenue")
    else:
        print(f"  [INSIGHT] Revenue may decrease despite inelastic classification")



[SIMULATION 1] Seafood
  Current Average Price: Rs141.84
  New Price (+ 5%):      Rs148.93
  Elasticity:            0.864
  Expected Qty Change:   +4.32%
  Expected Revenue Change: +9.54%
  Estimated ₹ Impact:    Rs+14,199.44
  [INSIGHT] Price increase likely to improve revenue

[SIMULATION 2] Breads
  Current Average Price: Rs140.95
  New Price (+ 5%):      Rs148.00
  Elasticity:            0.968
  Expected Qty Change:   +4.84%
  Expected Revenue Change: +10.08%
  Estimated ₹ Impact:    Rs+55,782.99
  [INSIGHT] Price increase likely to improve revenue

[SIMULATION 3] Soft Drinks
  Current Average Price: Rs131.49
  New Price (+ 5%):      Rs138.07
  Elasticity:            0.898
  Expected Qty Change:   +4.49%
  Expected Revenue Change: +9.72%
  Estimated ₹ Impact:    Rs+86,762.36
  [INSIGHT] Price increase likely to improve revenue


In [43]:
#calculating total impact in top 3 categries
if inelastic.shape[0] > 0:  
    total_potential = sum(
        row['total_sales'] * ((1.05) * (1 + row['elasticity'] * 0.05) - 1)
        for _, row in inelastic.head(simulation_count).iterrows()
    )
    
    print(f"\n[TOTAL OPPORTUNITY] Estimated revenue increase from pricing optimization:")
    print(f"                    Rs {total_potential:+,.2f} across top {simulation_count} categories")
else:
    print("\n[NOTE] No simulation performed - no inelastic categories identified")



[TOTAL OPPORTUNITY] Estimated revenue increase from pricing optimization:
                    Rs +156,744.80 across top 3 categories


In [31]:
print(f"\n{'='*70}")
print("MODEL QUALITY ASSESSMENT")
print(f"{'='*70}")

# Check statistical significance of results
significant = elasticity_df[elasticity_df['p_value'] < 0.05]
pct_significant = len(significant) / len(elasticity_df) * 100

print(f"\n[VALIDATION] {len(significant)}/{len(elasticity_df)} categories ({pct_significant:.1f}%) have")
print(f"             statistically significant elasticity estimates (p < 0.05)")

if pct_significant < 50:
    print(f"[WARNING] Low significance rate - consider collecting more data")
else:
    print(f"[QUALITY]  Majority of estimates are statistically reliable")

# Check model fit quality
avg_r_squared = elasticity_df['r_squared'].mean()
print(f"\n[MODEL FIT] Average R² across categories: {avg_r_squared:.3f}")

if avg_r_squared < 0.3:
    print(f"[NOTE] Moderate explanatory power - other factors (promotions, seasonality)")
    print(f"       also influence sales beyond price")
else:
    print(f"[QUALITY]  Price explains substantial variation in sales")


MODEL QUALITY ASSESSMENT

[VALIDATION] 16/16 categories (100.0%) have
             statistically significant elasticity estimates (p < 0.05)
[QUALITY]  Majority of estimates are statistically reliable

[MODEL FIT] Average R² across categories: 0.282
[NOTE] Moderate explanatory power - other factors (promotions, seasonality)
       also influence sales beyond price
